***Voter Inequality in Canada***

In [1]:
def read_file(filename):
    f = open(filename, 'r')
    abc = f.readlines()
    f.close()
    return abc

def tab_separate(contents):
    xyz = []
    for a in contents:
        b = a.split("\t")
        xyz.append(b)
    return xyz

def remove_non_data_lines(sep_contents):
    a1 = []
    for x in sep_contents:
        if len([y for y in x if y == '']) < 2:
            a1.append(x)
    return a1

def get_riding_names(clean_contents):
    ridings = []
    for i in range(338):
        ridings.append(clean_contents[i*3+1][0])
    return ridings

In [2]:
abc = read_file('current_projections.txt')
xyz = tab_separate(abc)
a1 = remove_non_data_lines(xyz)
ridings = get_riding_names(a1)

In [3]:
import fednum_dict as fed

In [4]:
a2 = {v: k for k, v in fed.fednum_dict.items()} 

In [5]:
found = {}
found['Haliburton – Kawaratha Lakes – Brock'] = 35034
found['Ahuntsic – Cartierville'] = 24003
found['Laval-Les \xceles'] = 24040 
found['Mont-Royal'] = 24052 
found["Ville-Marie – Le Sud-Ouest – L'Île-des-Soeurs"] = 24077 
found["Beauport – Côte-de-Beaupré – Île d'Orléans – Charlevoix"] = 24020
found['Portneuf – Jacques Cartier'] = 24058
found["South Shore – St. Margaret's"] = 12009

In [6]:
not_found = []
for r in ridings:
    if r not in found:
        if r in a2:
            found[r] = a2[r]
        else:
            new = r.strip('*')
            if new in a2:
                found[r] = a2[new]
            else:
                not_found.append(r)

In [7]:
not_found

[]

In [8]:
len(found)

338

In [9]:
len(not_found)

0

In [10]:
len(set(found.keys()))

338

In [11]:
len(set(found.values()))

338

In [12]:
import ridingForecast

In [13]:
ridingList = ridingForecast.parseCleanForecasts(a1)

In [14]:
print(ridingList[0])

Abbotsford
Winner: Conservative prob: 0.86

Conservative Party, low = 41.8, forecast = 44.0, high = 48.4
Liberal Party, low = 22.0, forecast = 24.4, high = 26.1
New Democratic Party, low = 22.8, forecast = 23.7, high = 25.6
Bloc Quebecois Party, low = 0.0, forecast = 0.0, high = 0.0
Green Party, low = 0.0, forecast = 0.0, high = 0.0
Other Party, low = 6.9, forecast = 7.7, high = 8.2
